## Project: Text Classification with RNN
### Goal

- Build a model that classifies movie reviews as positive or negative.  

- You’ll load a dataset  

- Preprocess text  

- Build an RNN (LSTM)  

- Train & evaluate  

- Test sample sentences  

## Dataset  
We’ll use a small built-in dataset (so no downloads).

nltk comes with a sample movie review corpus — perfect for hands-on NLP.

## 0) Install dependencies

In [ ]:
!pip install jupyter lab

In [1]:
!pip install torch torchvision nltk numpy

  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ---------------------------------------- 0.0/111.0 MB ? eta -:--:--
   - -------------------------------------- 3.4/111.0 MB 18.3 MB/s eta 0:00:06
   -- ------------------------------------- 8.1/111.0 MB 20.1 MB/s eta 0:00:06
   ---- ----------------------------------- 13.4/111.0 MB 21.5 MB/s eta 0:00:05
   ------ --------------------------------- 17.3/111.0 MB 21.0 MB/s eta 0:00:05
   ------- -------------------------------- 21.2/111.0 MB 20.3 MB/s eta 0:00:05
   --------- ------------------------------ 25.4/111.0 MB 20.1 MB/s eta 0:00:05
   ---------- ----------------------------- 29.4/111.0 MB 20.0 MB/s eta 0:00:05
   ------------ --------------------------- 34.1/111.0 MB 20.4 MB/s eta 0:00:04
   ------------- -------------------------- 38.0/111.0 MB 20.3 MB/s eta 0:00:04
   --------------- ------------------------ 41.9/111.0 MB 20.2 MB/s eta 0:00:04
   

## 1) Load sample dataset from NLTK

In [9]:
import nltk

# Download movie reviews (one time)
nltk.download('movie_reviews')

[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\dbwnd\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


True

## 2) Prepare Data

In [10]:
from nltk.corpus import movie_reviews
import random

# Build data list: (text, label)
data = []
for fileid in movie_reviews.fileids():
    text = movie_reviews.words(fileid)
    label = movie_reviews.categories(fileid)[0]
    data.append((" ".join(text), label))

# Shuffle
random.shuffle(data)

# Split
train_data = data[:1500]
test_data  = data[1500:]

print("Train size:", len(train_data))
print("Test size:", len(test_data))

Train size: 1500
Test size: 500


## 3) Tokenizer & Vocabulary

In [11]:
import torch
from collections import Counter

# Tokenize train texts
all_tokens = [word.lower() for text, _ in train_data for word in text.split()]
vocab = Counter(all_tokens)

# Keep top N words
vocab_size = 5000
most_common = vocab.most_common(vocab_size)
itos = [w for w, _ in most_common]
stoi = {w: i+1 for i, w in enumerate(itos)}  # reserve 0 for padding

print("Vocab size:", len(stoi))

Vocab size: 5000


## 4) Encode Examples & Pads

In [12]:
def encode_sentence(sent, max_len=200):
    tokens = sent.lower().split()
    encoded = [stoi.get(w, 0) for w in tokens[:max_len]]
    padded  = encoded + [0]*(max_len-len(encoded))
    return torch.tensor(padded)

## 5) Dataset + DataLoader

In [13]:
from torch.utils.data import Dataset, DataLoader

class MovieDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text, label = self.data[idx]
        x = encode_sentence(text)
        y = 1 if label == "pos" else 0
        return x, torch.tensor(y)

train_loader = DataLoader(MovieDataset(train_data), batch_size=32, shuffle=True)
test_loader  = DataLoader(MovieDataset(test_data),  batch_size=32)


## 6) RNN Model (LSTM)

In [14]:
import torch.nn as nn

class SentimentRNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_size):
        super().__init__()
        self.embed = nn.Embedding(vocab_size+1, embed_dim, padding_idx=0)
        self.lstm = nn.LSTM(embed_dim, hidden_size, batch_first=True)
        self.fc   = nn.Linear(hidden_size, 1)
        self.act  = nn.Sigmoid()

    def forward(self, x):
        emb = self.embed(x)
        out,_ = self.lstm(emb)
        out = out[:, -1]          # last hidden
        out = self.fc(out)
        return self.act(out)

model = SentimentRNN(vocab_size, embed_dim=128, hidden_size=128)


## 7) Train Loop

In [ ]:
import torch.optim as optim

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(5):
    total_loss = 0
    model.train()
    for x, y in train_loader:
        pred = model(x)
        loss = criterion(pred.squeeze(), y.float())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    print(f"Epoch {epoch+1}, loss {total_loss/len(train_loader):.4f}")


Epoch 1, loss 0.0123
Epoch 2, loss 0.0222
Epoch 3, loss 0.0153
Epoch 4, loss 0.0082
Epoch 5, loss 0.0083
Epoch 6, loss 0.0045
Epoch 7, loss 0.0035
Epoch 8, loss 0.0023
Epoch 9, loss 0.0062
Epoch 10, loss 0.0164
Epoch 11, loss 0.0051
Epoch 12, loss 0.0021
Epoch 13, loss 0.0135
Epoch 14, loss 0.0084
Epoch 15, loss 0.0046
Epoch 16, loss 0.0035
Epoch 17, loss 0.0029
Epoch 18, loss 0.0026
Epoch 19, loss 0.0022
Epoch 20, loss 0.0020
Epoch 21, loss 0.0021
Epoch 22, loss 0.0019
Epoch 23, loss 0.0035
Epoch 24, loss 0.0041
Epoch 25, loss 0.0054
Epoch 26, loss 0.0041
Epoch 27, loss 0.0029
Epoch 28, loss 0.0027
Epoch 29, loss 0.0021
Epoch 30, loss 0.0009
Epoch 31, loss 0.0041
Epoch 32, loss 0.0056
Epoch 33, loss 0.0302
Epoch 34, loss 0.0304
Epoch 35, loss 0.0125
Epoch 36, loss 0.0079
Epoch 37, loss 0.0063
Epoch 38, loss 0.0035
Epoch 39, loss 0.0027
Epoch 40, loss 0.0019
Epoch 41, loss 0.0022
Epoch 42, loss 0.0085
Epoch 43, loss 0.0041
Epoch 44, loss 0.0037
Epoch 45, loss 0.0034
Epoch 46, loss 0.00

## 8) Evaluate

In [20]:
model.eval()
correct = 0
total   = 0
with torch.no_grad():
    for x, y in test_loader:
        pred = model(x)
        predicted = (pred.squeeze()>0.5).long()
        correct += (predicted == y).sum().item()
        total += y.size(0)

print("Test Accuracy:", correct/total)

Test Accuracy: 0.504


## 9) Try Custom Sentence

In [24]:
def predict_sent(sent):
    x = encode_sentence(sent).unsqueeze(0)  # batch=1
    with torch.no_grad():
        pred = model(x).item()
    return "Positive" if pred > 0.5 else "Negative"

print(predict_sent("I loved this movie!"))
print(predict_sent("This was the worst film ever"))

Negative
Negative


# 성능 개선

In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from nltk.corpus import movie_reviews
from collections import Counter
import random
import nltk

# 1. 데이터 준비 및 전처리 (NLTK 데이터 로드)
nltk.download('movie_reviews')

data = []
for fileid in movie_reviews.fileids():
    # 간단한 정제: 알파벳만 남기고 소문자로 변환
    words = [word.lower() for word in movie_reviews.words(fileid) if word.isalpha()]
    label = movie_reviews.categories(fileid)[0]
    data.append((" ".join(words), label))

random.shuffle(data)
train_data = data[:1600] # 데이터 비중 조절
test_data  = data[1600:]

# 2. 단어 사전(Vocab) 구축
all_tokens = [word for text, _ in train_data for word in text.split()]
vocab = Counter(all_tokens)
vocab_size = 8000 # 단어장 크기 확대
itos = [w for w, _ in vocab.most_common(vocab_size)]
stoi = {w: i+2 for i, w in enumerate(itos)} # 0: padding, 1: unknown
stoi['<PAD>'] = 0
stoi['<UNK>'] = 1

# 3. 인코딩 함수 (문장 길이를 300으로 상향)
def encode_sentence(sent, max_len=300):
    tokens = sent.split()
    encoded = [stoi.get(w, 1) for w in tokens[:max_len]]
    padded  = encoded + [0]*(max_len - len(encoded))
    return torch.tensor(padded)

class MovieDataset(Dataset):
    def __init__(self, data):
        self.data = data
    def __len__(self): return len(self.data)
    def __getitem__(self, idx):
        text, label = self.data[idx]
        return encode_sentence(text), torch.tensor(1 if label == "pos" else 0)

train_loader = DataLoader(MovieDataset(train_data), batch_size=64, shuffle=True)
test_loader  = DataLoader(MovieDataset(test_data),  batch_size=64)

# 4. 고도화된 모델 설계 (Bi-LSTM)
class AdvancedRNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size + 2, embed_dim, padding_idx=0)
        # bidirectional=True 추가로 성능 강화
        self.lstm = nn.LSTM(embed_dim, hidden_size, batch_first=True, bidirectional=True, num_layers=2, dropout=0.5)
        # 양방향이므로 hidden_size에 2를 곱함
        self.fc = nn.Linear(hidden_size * 2, 1)
        self.dropout = nn.Dropout(0.5)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        embedded = self.dropout(self.embedding(x))
        lstm_out, (hidden, cell) = self.lstm(embedded)
        
        # 마지막 타임스텝의 양방향 hidden state를 합침
        last_hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)
        out = self.fc(last_hidden)
        return self.sigmoid(out)

# 모델 생성 (임베딩 128, 은닉층 128)
model = AdvancedRNN(vocab_size, 128, 128)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss()

# 5. 학습 루프
print("학습 시작...")
for epoch in range(10): # 에폭 증가
    model.train()
    total_loss = 0
    for x, y in train_loader:
        optimizer.zero_grad()
        pred = model(x).squeeze()
        loss = criterion(pred, y.float())
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}: Loss = {total_loss/len(train_loader):.4f}")

# 6. 평가
model.eval()
correct = 0
with torch.no_grad():
    for x, y in test_loader:
        pred = (model(x).squeeze() > 0.5).long()
        correct += (pred == y).sum().item()

print(f"\n최종 테스트 정확도: {correct/len(test_data) * 100:.2f}%")

[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\dbwnd\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


학습 시작...
Epoch 1: Loss = 0.6951
Epoch 2: Loss = 0.6863
Epoch 3: Loss = 0.6662
Epoch 4: Loss = 0.6404
Epoch 5: Loss = 0.6108
Epoch 6: Loss = 0.5759
Epoch 7: Loss = 0.5546
Epoch 8: Loss = 0.4994
Epoch 9: Loss = 0.4459
Epoch 10: Loss = 0.4510

최종 테스트 정확도: 56.50%


In [29]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from nltk.corpus import movie_reviews
import random
import nltk
import re
from collections import Counter

# 1. 데이터 로드 및 균형 잡기 (개선 포인트)
nltk.download('movie_reviews')

# 긍정/부정 데이터를 각각 분리해서 가져옵니다.
pos_fileids = movie_reviews.fileids('pos')
neg_fileids = movie_reviews.fileids('neg')

data = []
# 긍정 데이터 1000개, 부정 데이터 1000개를 균형 있게 수집
for fileid in pos_fileids:
    words = [word.lower() for word in movie_reviews.words(fileid) if word.isalpha()]
    data.append((" ".join(words), "pos"))
for fileid in neg_fileids:
    words = [word.lower() for word in movie_reviews.words(fileid) if word.isalpha()]
    data.append((" ".join(words), "neg"))

# 데이터를 섞고 학습/테스트용으로 분할 (8:2 비율)
random.shuffle(data)
train_data = data[:1600]
test_data  = data[1600:]

# 2. 단어 사전(Vocab) 구축 (개선 포인트: 단어장 크기 확대)
all_tokens = [word for text, _ in train_data for word in text.split()]
vocab = Counter(all_tokens)
vocab_size = 10000 
itos = [w for w, _ in vocab.most_common(vocab_size)]
stoi = {w: i+2 for i, w in enumerate(itos)} 
stoi['<PAD>'] = 0
stoi['<UNK>'] = 1

# 3. 데이터셋 및 인코딩 (문장 길이를 늘려 정보 손실 방지)
def encode_sentence(sent, max_len=400):
    tokens = sent.split()
    encoded = [stoi.get(w, 1) for w in tokens[:max_len]]
    padded  = encoded + [0]*(max_len - len(encoded))
    return torch.tensor(padded)

class MovieDataset(Dataset):
    def __init__(self, data):
        self.data = data
    def __len__(self): return len(self.data)
    def __getitem__(self, idx):
        text, label = self.data[idx]
        return encode_sentence(text), torch.tensor(1 if label == "pos" else 0)

train_loader = DataLoader(MovieDataset(train_data), batch_size=32, shuffle=True)
test_loader  = DataLoader(MovieDataset(test_data),  batch_size=32)

# 4. 모델 설계 (양방향 LSTM + Dropout)
class SentimentBiLSTM(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size + 2, embed_dim, padding_idx=0)
        self.lstm = nn.LSTM(embed_dim, hidden_size, batch_first=True, bidirectional=True, num_layers=2, dropout=0.5)
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(hidden_size * 2, 1) # Bidirectional이라서 *2
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        emb = self.dropout(self.embedding(x))
        _, (hidden, _) = self.lstm(emb)
        # 마지막 층의 양방향 hidden state를 합침
        cat_hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)
        out = self.fc(self.dropout(cat_hidden))
        return self.sigmoid(out)

# 모델 초기화
model = SentimentBiLSTM(vocab_size, 128, 128)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 5. 학습 루프 (에폭 증가)
print("학습을 시작합니다...")
for epoch in range(15):
    model.train()
    epoch_loss = 0
    for x, y in train_loader:
        optimizer.zero_grad()
        pred = model(x).squeeze()
        loss = criterion(pred, y.float())
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f"Epoch {epoch+1}/15, Loss: {epoch_loss/len(train_loader):.4f}")

# 6. 실제 데이터 테스트 함수
def predict_review(sentence):
    model.eval()
    # 전처리
    sentence_clean = re.sub(r'[^a-zA-Z\s]', '', sentence).lower()
    encoded = encode_sentence(sentence_clean, max_len=400).unsqueeze(0)
    
    with torch.no_grad():
        prob = model(encoded).item()
    
    sentiment = "긍정(Positive) 😊" if prob > 0.5 else "부정(Negative) ☹️"
    score = prob * 100 if prob > 0.5 else (1 - prob) * 100
    print(f"리뷰: \"{sentence}\"")
    print(f"결과: {sentiment} (확신도: {score:.2f}%)")
    print("-" * 30)

# 테스트 실행
print("\n[새로운 데이터 테스트 결과]\n")
predict_review("This movie was an absolute masterpiece with great acting")
predict_review("I fell asleep in the middle of the film it was so boring")
predict_review("The visual effects were stunning but the story was weak")

[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\dbwnd\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


학습을 시작합니다...
Epoch 1/15, Loss: 0.6932
Epoch 2/15, Loss: 0.6875
Epoch 3/15, Loss: 0.6720
Epoch 4/15, Loss: 0.6469
Epoch 5/15, Loss: 0.5863
Epoch 6/15, Loss: 0.5588
Epoch 7/15, Loss: 0.5039
Epoch 8/15, Loss: 0.4388
Epoch 9/15, Loss: 0.4273
Epoch 10/15, Loss: 0.4042
Epoch 11/15, Loss: 0.3274
Epoch 12/15, Loss: 0.3009
Epoch 13/15, Loss: 0.2463
Epoch 14/15, Loss: 0.2484
Epoch 15/15, Loss: 0.2253

[새로운 데이터 테스트 결과]

리뷰: "This movie was an absolute masterpiece with great acting"
결과: 부정(Negative) ☹️ (확신도: 94.47%)
------------------------------
리뷰: "I fell asleep in the middle of the film it was so boring"
결과: 부정(Negative) ☹️ (확신도: 94.24%)
------------------------------
리뷰: "The visual effects were stunning but the story was weak"
결과: 부정(Negative) ☹️ (확신도: 92.83%)
------------------------------


In [30]:
predict_review("I loved this movie!")
predict_review("This was the worst film ever")

리뷰: "I loved this movie!"
결과: 부정(Negative) ☹️ (확신도: 98.72%)
------------------------------
리뷰: "This was the worst film ever"
결과: 부정(Negative) ☹️ (확신도: 99.22%)
------------------------------
